In [1]:
import math
from typing import List, Optional, Tuple, Union
from abc import ABC, abstractmethod

import datasets
import torch
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
import logging
import copy
import gc

from datasets import load_dataset
from tqdm import tqdm
from transformers import (
    PreTrainedModel,
    PretrainedConfig,
    AutoConfig,
    AutoModelForCausalLM,
    LlamaForCausalLM,
    LlamaConfig,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    HfArgumentParser
)

from transformers import (
    HfArgumentParser,
    TrainingArguments,
    Trainer
)

# from modeling_qwen2 import (
#     Qwen2RMSNorm, 
#     Qwen2RotaryEmbedding, 
#     Qwen2MLP, 
#     Qwen2Attention, 
#     Qwen2FlashAttention2, 
#     Qwen2SdpaAttention, 
#     Qwen2DecoderLayer, 
#     Qwen2PreTrainedModel, 
#     Qwen2Model, 
#     Qwen2ForCausalLM,
# )

from configuration_qwen2 import Qwen2Config

from transformers.modeling_outputs import (
    BaseModelOutputWithPast,
    CausalLMOutputWithPast
)

# Configure logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [2]:
rewrite_smol = load_dataset("HuggingFaceTB/smoltalk", "smol-rewrite", split="test")
summarize_smol = load_dataset("HuggingFaceTB/smoltalk", "smol-summarize", split="test")

In [3]:
rewrite_train = load_dataset("HuggingFaceTB/smoltalk", "smol-rewrite", split="train")
summarize_train = load_dataset("HuggingFaceTB/smoltalk", "smol-summarize", split="train")

In [4]:
# import os
# os.listdir("/workspace/HUB_LLM/091224_llama3_70b/")

In [5]:
from transformers import GenerationConfig, TextStreamer
def generate(prompt, model, tokenizer, max_new_tokens=1024):
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    model.eval()
    with torch.no_grad():
        generation_config = GenerationConfig(
            repetition_penalty=1.13,
            max_new_tokens=max_new_tokens,
            temperature=0.4,
            top_p=0.95,
            # top_k=20,
            # bos_token_id=tokenizer.bos_token_id,
            # eos_token_id=tokenizer.eos_token_id,
            # eos_token_id=0, # for open-end generation.
            eos_token_id=[128001, 128009],
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False,
            use_cache=True,
            return_dict_in_generate=True,
            output_attentions=False,
            output_hidden_states=False,
            output_scores=False,
        )
        streamer = TextStreamer(tokenizer, skip_prompt=True)
        generated = model.generate(
            inputs=input_ids,
            generation_config=generation_config,
            streamer=streamer,
        )
    gen_tokens = generated["sequences"].cpu()[:, len(input_ids[0]):]
    output = tokenizer.batch_decode(gen_tokens)[0]
    output = output.split(tokenizer.eos_token)[0]
    return output.strip()

def get_logits(text, model, tokenizer):
    input_ids = tokenizer(text, return_tensors="pt").to(model.device)
    model.eval()
    with torch.no_grad():
        logits = model(**input_ids).logits
    return logits

def get_hidden_states(text, model, tokenizer):
    input_ids = tokenizer(text, return_tensors="pt").to(model.device)
    model.eval()
    with torch.no_grad():
        outputs = model(**input_ids, output_hidden_states=True, use_cache=False)
    return outputs

In [6]:
def free_memory():
    if not torch.cuda.is_available():
        logger.info("CUDA is not available. No GPU memory to free.")
        return
        
    initial_memory = torch.cuda.memory_allocated()
    logger.info(f"Initial GPU memory allocated: {initial_memory / 1024**3:.2f} GB")
    gc.collect()
    torch.cuda.empty_cache()

    final_memory = torch.cuda.memory_allocated()
    logger.info(f"Final GPU memory allocated: {final_memory / 1024**3:.2f} GB")

    freed_memory = initial_memory - final_memory
    logger.info(f"Freed GPU memory: {freed_memory / 1024**3:.2f} GB")

In [7]:
class MaskConfig(PretrainedConfig):
    def __init__(
        self,
        mode: str = None,
        value: Union[float, torch.Tensor] = None,
        size: torch.Size = None,
        **kwargs,
    ):
        self.mode = mode
        self.value = value
        self.size = size
        super().__init__(**kwargs)

class Mask(nn.Module):
    def __init__(self, mask_config: MaskConfig):
        super().__init__()
        self.config = mask_config
        self.size = mask_config.size
        assert self.size is not None, "Mask size must be specified."

        value = mask_config.value
        if value is not None:
            logger.warning("Highly reccommend initializing mask value using dedicated setup functions.")
            if not isinstance(value, torch.Tensor): 
                try: value = torch.tensor(value)
                except: raise ValueError(
                    f"Unable to convert {value} to torch.Tensor required for initializing a mask's weight."
                )

        ## TODO: might refactor later: modify _get_ones() to handle scalar mode.
        if mask_config.mode == "scalar":
            self.weight = nn.Parameter(value if value is not None else torch.tensor(1.0))
            
        elif mask_config.mode in ("vector_input", "vector_output"):
            ones = self._get_ones(mask_config.mode)
            self.weight = nn.Parameter(value if value is not None else ones)
        else:
            raise ValueError(f"Unsupported mask mode: {mask_config.mode}")

        self._check_shape_compatibility()

    def _get_ones(self, mode: str) -> torch.Tensor:
        """Generates a tensor of ones based on mode and size."""
        dim = 0 if mode == "vector_output" else -1
        features = self.size[dim]
        if len(self.size) == 2 and mode == "vector_output":
            return torch.ones(features, 1)
        else:
            return torch.ones(features)
          

    def _check_shape_compatibility(self):
        """Raises ValueError if the mask shape is incompatible with its size."""
        try:
            in_test = torch.rand(self.size)
            out_test = self.weight * in_test
            assert out_test.shape == in_test.shape, (
                "After applying mask, the shape of input weight does not stay the same."
            )
        except RuntimeError:
            raise ValueError("Mask initialized with an incompatible shape.")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.size != x.shape:
            logger.warning("Warning: Input shape does not match mask shape.")
        return x * self.weight

In [8]:
class ModuleWithMask(nn.Module, ABC):
    def __init__(self, *args, **kwargs):
        super(ModuleWithMask, self).__init__()

    @abstractmethod
    def forward(self, x):
        pass

class ModulesWithMasks(nn.Module, ABC):
    def __init__(self, *args, **kwargs):
        super(ModulesWithMasks, self).__init__()

    @abstractmethod
    def forward(self, x):
        pass
        
    @abstractmethod
    def get_raw_masks(self):
        pass
        
    @abstractmethod
    def get_constrained_masks(self):
        pass

In [9]:
def relu_01(x: torch.Tensor) -> torch.Tensor:
    return F.relu(x) - F.relu(x - 1.0)

relu_01(torch.tensor(0.5))

tensor(0.5000)

In [10]:
def silu_01(x: torch.Tensor) -> torch.Tensor:
    return F.silu(x) - F.silu(x - 1.0)

def relu_01(x: torch.Tensor) -> torch.Tensor:
    return F.relu(x) - F.relu(x - 1.0)

def normalize(v, dim, eps: float = 1e-8):
    norm_v = torch.linalg.norm(v, dim=dim)
    norm_v[norm_v < eps] = 1.0
    v = v / norm_v
    return v

class Constrainer(nn.Module):

    def __init__(self, component_weights, constrain_mode, mask_mode):
        super().__init__()
        self.statistics = None
        self.constrain_mode = constrain_mode
        self.mask_mode = mask_mode
        if self.constrain_mode not in ("identity", "01", "-11", "spherical", "sum1"):
            raise ValueError(f"Does not support {self.constrain_mode} constraint yet!")
            
        if (self.constrain_mode == "spherical" and all([w is not None for w in component_weights])):
            self._get_spherical_stats(component_weights)

    def _get_spherical_stats(
        self, 
        component_weights: List[torch.Tensor], 
        DOT_THRESHOLD: float = 0.99995
    ):
        with torch.no_grad():
            if any([w is None for w in component_weights]):
                raise ValueError("Spherical constraint (SLERP) does not support None weights.")
            if len(component_weights) != 2: 
                raise ValueError(
                    "Spherical constraint (SLERP) only supports 2 component weights, " +
                    f"{len(component_weights)} components found."
                )

            
            dim = 0 if self.mask_mode in ("vector_input") else None
            v0 = normalize(component_weights[0], dim=dim)
            v1 = normalize(component_weights[1], dim=dim)
            self.dots = torch.sum(v0 * v1, dim=dim, keepdim=False) ## (out_features, in_features) -> (in_features, )
            self.theta_0s = torch.arccos(self.dots)
            self.sin_theta_0s = torch.sin(self.theta_0s)
        
    def _constrain_identity(self, mask_weights: List[torch.Tensor]):
        return mask_weights

    def _constrain_sumone(self, mask_weights: List[torch.Tensor]):
        W = [w / sum(mask_weights) for w in mask_weights]
        return W

    def _constrain_0_1(self, mask_weights: List[torch.Tensor]):
        W = [torch.sigmoid(w) for w in mask_weights]
        return W

    def _constrain_neg1_1(self, mask_weights: List[torch.Tensor]):
        return mask_weights

    def _constrain_spherical(self, mask_weights: List[torch.Tensor], DOT_THRESHOLD: float = 0.9995):
        assert len(mask_weights) == 2, (
            "Spherical constraint (SLERP) only supports 2 mask weights"
        )
        
        # Transform raw masks to factor t's.
        ## QUESTION MASK: Does this modify mask_weights in-place? I only 
        ## update them via backprop.
        # W = [torch.exp(w) for w in mask_weights]
        # T = W[0] / sum(W)

        ## ignore mask_weights[1]
        T = silu_01(mask_weights[0])

        # Angle at timestep t's
        theta_ts = self.theta_0s * T
        sin_theta_ts = torch.sin(theta_ts)

        # Finish calculating slerp factors
        S0 = torch.sin(self.theta_0s - theta_ts) / self.sin_theta_0s
        S1 = sin_theta_ts / self.sin_theta_0s

        # Avoid NaN
        nan_indices = self.dots.float() > DOT_THRESHOLD
        S0[nan_indices] = 1 - T[nan_indices]
        S1[nan_indices] = T[nan_indices]
        
        return [S0, S1]
        
    def forward(self, mask_weights: List[torch.Tensor]):
        if any([w is None for w in mask_weights]):
            return mask_weights
            
        if self.constrain_mode == "identity":
            return self._constrain_identity(mask_weights)
        elif self.constrain_mode == "01":
            return self._constrain_0_1(mask_weights)
        elif self.constrain_mode == "-11":
            return self._constrain_neg1_1(mask_weights)
        elif self.constrain_mode == "sum1":
            return self._constrain_sumone(mask_weights)
        elif self.constrain_mode == "spherical":
            return self._constrain_spherical(mask_weights)
        else:
            raise ValueError(f"Does not support {self.constrain_mode} constraint yet!")

In [11]:
class LinearsWithMasks(ModulesWithMasks):
    def __init__(
        self,
        linears: List[nn.Linear],
        weight_modes: List[str] = None,
        weight_values: List[float] = None,
        bias_modes: List[str] = None,
        bias_values: List[float] = None,
        constrain_mode: str = "identity",
    ):
        super().__init__()

        if not all(isinstance(linear, nn.Linear) for linear in linears):
            raise ValueError("All elements in 'linears' must be instances of nn.Linear.")

        if weight_values is None or len(weight_values) != len(linears):
            raise ValueError(
                f"Weight values for masks: {weight_values} do not match with linear layers: {linears}"
            )
        if bias_values is None:
            bias_values = [None] * len(linears)
        if len(bias_values) != len(linears):
            raise ValueError(
                f"Bias values for masks: {bias_values} do not match with linear layers: {linears}"
            )

        self.linears = nn.ModuleList(linears)
        self.constrain_mode = constrain_mode

        self.weight_masks = nn.ModuleList([
            Mask(MaskConfig(mode, value, linear.weight.shape))
            for mode, value, linear in zip(weight_modes, weight_values, linears)
        ])
        
        self.weight_masks_constrainer = Constrainer(
            component_weights=[x.weight for x in self.linears], 
            constrain_mode=constrain_mode, mask_mode=weight_modes[0]
        )

        self.bias_masks = nn.ModuleList([
            Mask(MaskConfig(mode, value, linear.bias.shape)) if linear.bias is not None else None
            for mode, value, linear in zip(bias_modes, bias_values, linears)
        ])
        
        self.bias_masks_constrainer = Constrainer(
            component_weights=[x.bias if x.bias is not None else None for x in self.linears],
            constrain_mode=constrain_mode, mask_mode=bias_modes[0]
        )

    def forward(self, x):
        constrained_weight_masks = self.weight_masks_constrainer([m.weight for m in self.weight_masks])
        masked_weights = [
            w_mask * linear.weight for w_mask, linear in zip(constrained_weight_masks, self.linears)
        ]
        merged_weight = sum(masked_weights)

        constrained_bias_masks = self.bias_masks_constrainer(
            [m.weight if m is not None else None for m in self.bias_masks]
        )
        masked_biases = [
            b_mask * linear.bias if linear.bias is not None and b_mask is not None else linear.bias
            for b_mask, linear in zip(constrained_bias_masks, self.linears)
        ]

        merged_bias = (
            sum(b if b is not None else torch.zeros_like(merged_weight[:, 0]) for b in masked_biases)
            if not all(b is None for b in masked_biases) else None
        )

        return nn.functional.linear(x, merged_weight, merged_bias)

    def get_raw_masks(self):
        with torch.no_grad():
            return {
                "weight_masks": [m.weight for m in self.weight_masks],
                "bias_masks": [m.weight if m is not None else None for m in self.bias_masks],
            }

    def get_constrained_masks(self):
        with torch.no_grad():
            constrained_weight_masks = self.weight_masks_constrainer(
                [m.weight for m in self.weight_masks]
            )
            constrained_bias_masks = self.bias_masks_constrainer(
                [m.weight if m is not None else None for m in self.bias_masks]
            )
            return {
                "weight_masks": constrained_weight_masks,
                "bias_masks": constrained_bias_masks,
            }

In [12]:
class RMSNormsWithMasks(ModulesWithMasks):
    def __init__(
        self,
        rms_norms: List[nn.Module],
        modes: List[str] = None,
        values: List[float] = None,
        constrain_mode: str = "identity",
    ):
        super().__init__()
        sizes = [norm.weight.shape for norm in rms_norms]
        if any([mode != "scalar" for mode in modes]):
            logger.warning(
                f"Though you want to make a masks of modes {modes} " + \
                "for RMSNorms' weights, by default a mask only accepts a scalar mask. " + \
                "Converting modes to `scalar`."
            )
            modes = ["scalar"] * len(modes)
            
        if values is None or len(values) != len(rms_norms):
            raise ValueError(f"values for masks: {values} do not match with RMSNorm layers: {rms_norms}")

        self.rms_norms = nn.ModuleList(rms_norms)
        self.masks = nn.ModuleList([
            Mask(MaskConfig(mode, value, norm.weight.shape))
            for mode, value, norm in zip(modes, values, rms_norms)
        ])
        self.masks_constrainer = Constrainer(
            component_weights=[norm.weight for norm in self.rms_norms], 
            constrain_mode=constrain_mode, mask_mode=modes[0]
        )

    def forward(self, hidden_states):
        constrained_masks = self.masks_constrainer([m.weight for m in self.masks])
        masked_weights = [mask * norm.weight for mask, norm in zip(constrained_masks, self.rms_norms)]
        merged_weight = sum(masked_weights)
        variance_epsilon = self.rms_norms[0].variance_epsilon
        for norm in self.rms_norms:
            assert variance_epsilon == norm.variance_epsilon, ("Variance epsilon among models must be consistent")
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + variance_epsilon)
        return merged_weight * hidden_states.to(input_dtype)

    def get_raw_masks(self):
        with torch.no_grad():
            return {"masks": [m.weight for m in self.masks]}

    def get_constrained_masks(self):
        with torch.no_grad():
            constrained_masks = self.masks_constrainer([m.weight for m in self.masks])
            return {"masks": constrained_masks}

In [13]:
class EmbeddingsWithMasks(ModulesWithMasks):
    def __init__(
        self,
        embeddings: List[nn.Embedding],
        modes: List[str] = None,
        values: List[float] = None,
        constrain_mode: str = "identity"
    ):
        super().__init__()
        if values is None or len(values) != len(embeddings):
            raise ValueError(f"values for masks: {values} do not match with Embedding layers: {embeddings}")

        self.embeddings = nn.ModuleList(embeddings)
        sizes = [emb.weight.shape for emb in embeddings]
        self.masks = nn.ModuleList([
            Mask(MaskConfig(mode, value, size))
            for mode, value, size in zip(modes, values, sizes)
        ])
        self.masks_constrainer = Constrainer(
            component_weights=[emb.weight for emb in self.embeddings], 
            constrain_mode=constrain_mode, mask_mode=modes[0]
        )

    def forward(self, input_ids):
        constrained_masks = self.masks_constrainer([m.weight for m in self.masks])
        masked_weights = [mask * emb.weight for mask, emb in zip(constrained_masks, self.embeddings)]
        merged_weight = sum(masked_weights)
        
        an_embedding = self.embeddings[0]
        for other_embedding in self.embeddings:
            assert an_embedding.padding_idx == other_embedding.padding_idx
            assert an_embedding.max_norm == other_embedding.max_norm
            assert an_embedding.norm_type == other_embedding.norm_type
            assert an_embedding.scale_grad_by_freq == other_embedding.scale_grad_by_freq
            assert an_embedding.sparse == other_embedding.sparse
            
        return nn.functional.embedding(
            input_ids,
            merged_weight,
            padding_idx=an_embedding.padding_idx,
            max_norm=an_embedding.max_norm,
            norm_type=an_embedding.norm_type,
            scale_grad_by_freq=an_embedding.scale_grad_by_freq,
            sparse=an_embedding.sparse,
        )
        
    def get_raw_masks(self):
        with torch.no_grad():
            return {"masks": [m.weight for m in self.masks]}

    def get_constrained_masks(self):
        with torch.no_grad():
            constrained_masks = self.masks_constrainer([m.weight for m in self.masks])
            return {"masks": constrained_masks}

In [14]:
def odd_one_out(masked_module: nn.Module, selected_idx: int):  
    assert selected_idx is not None and isinstance(selected_idx, int), (
        "Must provide valid model index. Check whether passed index is `int`"
    )
    masks_modules = []
    for name, child in masked_module.named_children():
        if not isinstance(child, nn.ModuleList): continue
        assert selected_idx < len(child), (
            f"There are only {len(child)} component models, passed model index is {selected_idx}"
        )
        ## exclude sub_module that is None, aka bias_masks.
        if all(isinstance(sub_module, Mask) for sub_module in child):
            masks_modules.append(child)
        
    for masks in masks_modules:
        for i, mask in enumerate(masks):
            value = 1.0 if i == selected_idx else 0.0
            with torch.no_grad():
                mask.weight.data.fill_(value)

def random_init(masked_module: nn.Module):
    masks_modules = []
    for name, child in masked_module.named_children():
        if not isinstance(child, nn.ModuleList): continue
        ## exclude sub_module that is None, aka bias_masks.
        if all(isinstance(sub_module, Mask) for sub_module in child):
            masks_modules.append(child)
        
    for masks in masks_modules:
        for i, mask in enumerate(masks):
            with torch.no_grad():
                random_value = torch.rand_like(mask.weight.data)
                mask.weight.data = random_value

def uniform_init(masked_module: nn.Module, factors: List[float]):  
    masks_modules = []
    for name, child in masked_module.named_children():
        if not isinstance(child, nn.ModuleList): continue
        assert len(factors) == len(child), (
            f"There are {len(child)} component models, but your passed factors have {len(factors)} values."
        )
        ## exclude sub_module that is None, aka bias_masks.
        if all(isinstance(sub_module, Mask) for sub_module in child):
            masks_modules.append(child)

    for masks in masks_modules:
        for factor, mask in zip(factors, masks):
            with torch.no_grad():
                mask.weight.data.fill_(factor)

In [15]:
def find_masked_modules(module):
    masked_module_names = []
    for parent_name, parent_module in module.named_modules():
        for name, child in parent_module.named_children():
            full_child_name = f"{parent_name}.{name}" if parent_name else name
            if ("WithMasks" in type(child).__name__):
                masked_module_names.append(full_child_name)

    return masked_module_names

def get_init_method(strategy):
    MAP = {
        "random": random_init,
        "odd_one_out": odd_one_out,
        "uniform": uniform_init
    }
    selected_init_method = MAP[strategy]
    
    return selected_init_method
    
def set_masks(root_module, strategy="random", **kwargs):

    init_method = get_init_method(strategy)
    masked_module_names = find_masked_modules(root_module)
    
    for module_name in tqdm(masked_module_names, desc="Setting up masks"):
        module_names = module_name.split(".")
        target_module = root_module
        for m_name in module_names:
            target_module = getattr(target_module, m_name)

        init_method(target_module, **kwargs)

def set_constrainers(root_module, constrain_mode, **kwargs):
    if constrain_mode not in ("identity", "01", "-11", "spherical"):
        raise ValueError(f"Does not support {self.constrain_mode} constraint yet!")

In [16]:
class MergerConfig(PretrainedConfig):
    def __init__(
        self,
        model_paths: List[str] = None,
        mode: str = None,
        constrain_mode: str = None,
        **kwargs,
    ):
        self.model_paths = model_paths
        self.mode = mode
        self.constrain_mode = constrain_mode
        super().__init__(**kwargs)

class Merger(PreTrainedModel):
    def __init__(self, merge_config):
        super().__init__(merge_config)
        """
        Need to check whether models are mergeable (having some sort of the same config)
        """
        self.merge_config = merge_config
        self.num_models = len(merge_config.model_paths)
        self.configs = [
            AutoConfig.from_pretrained(path) 
            # Qwen2Config.from_pretrained(path)
            for path in merge_config.model_paths
        ]
        # self.merger = Qwen2ForCausalLM(self.config)
        self.models = nn.ModuleList([
            # Qwen2ForCausalLM.from_pretrained(
            AutoModelForCausalLM.from_pretrained(
                merge_config.model_paths[i], 
                config=self.configs[i],
                torch_dtype=torch.bfloat16
            ) 
            for i in range(self.num_models)
        ])
        # self.__post_init__(merge_config)
        
    def __post_init__(self):
        for model in self.models:
            for param in model.parameters():
                param.requires_grad = False
                
        self.merger = copy.deepcopy(self.models[0])
        init_masks(
            self.merger, self.models, self.merge_config
        )
        free_memory()
        
    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        cache_position: Optional[torch.LongTensor] = None,
        num_logits_to_keep: int = 0,
        **loss_kwargs,
    ) -> Union[Tuple, CausalLMOutputWithPast]:
        output_attentions = output_attentions if output_attentions is not None else True
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else True
        )
        return_dict = return_dict if return_dict is not None else True

        # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
        inputs = dict(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
        )
        return {
            "merger_outputs": self.merger(**inputs),
            "component_outputs": [model(**inputs) for model in self.models]
        }

In [17]:
def find_modules_to_add_masks(target_module):
    module_names_to_replace = []
    for parent_name, parent_module in target_module.named_modules():
        for name, child in parent_module.named_children():
            full_child_name = f"{parent_name}.{name}" if parent_name else name
            if (isinstance(child, (nn.Linear, nn.Embedding)) 
                or "RMSNorm" in type(child).__name__):
                module_names_to_replace.append(full_child_name)

    return module_names_to_replace

def init_masks(target_module: nn.Module, ref_modules: nn.Module, merge_config: MergerConfig):
    """
    Replaces eligible submodules in target_module with masked versions, 
    using corresponding modules from ref_modules as a reference for weights.

    Args:
        target_module: The module in which to replace submodules.
        ref_modules: A list of modules to use as a reference for weights.
        strategy: The initialization strategy for factors ("naive" or others to be implemented).
    """
    mode = merge_config.mode
    constrain_mode = merge_config.constrain_mode
    module_names_to_replace = find_modules_to_add_masks(target_module)
    
    for module_name in tqdm(module_names_to_replace, desc="Initializing masks"):
        module_names = module_name.split(".")
        target_child = target_module
        ref_children = ref_modules

        for m_name in module_names:
            target_child = getattr(target_child, m_name)
            ref_children = [getattr(ref_module, m_name) for ref_module in ref_children]

        num_components = len(ref_modules)
        modes = [mode for _ in ref_children]
        factors = [None for _ in ref_children]

        if isinstance(target_child, nn.Linear):
            new_module = LinearsWithMasks(
                linears=ref_children,
                weight_modes=modes,
                weight_values=factors,
                bias_modes=modes,
                bias_values=factors,
                constrain_mode=constrain_mode
            )

        elif isinstance(target_child, nn.Embedding):
            new_module = EmbeddingsWithMasks(ref_children, modes, factors, constrain_mode)
        elif "RMSNorm" in type(target_child).__name__:
            new_module = RMSNormsWithMasks(ref_children, modes, factors, constrain_mode)

        # Replace the original module with the new masked module
        parent_module = target_module
        for m_name in module_names[:-1]:
            parent_module = getattr(parent_module, m_name)
        setattr(parent_module, module_names[-1], new_module)

In [18]:
min(0.9, np.exp(1/np.e))

0.9

In [19]:
merge_config = MergerConfig(
    model_paths = [
        "/workspace/dont15/models/llama32_smol_rewrite_50k/",
        "/workspace/dont15/models/llama32_smol_summarize_50k/",
        # "/workspace/HUB_LLM/Llama-3.2-3B-Instruct",
    ],
    # mode = "vector_input",
    mode = "scalar",
    constrain_mode = "spherical"
)
merge_config

MergerConfig {
  "constrain_mode": "spherical",
  "mode": "scalar",
  "model_paths": [
    "/workspace/dont15/models/llama32_smol_rewrite_50k/",
    "/workspace/dont15/models/llama32_smol_summarize_50k/"
  ],
  "transformers_version": "4.47.1"
}

In [20]:
# merger.config = merge_config

In [21]:
merger = Merger(merge_config)
merger = merger.to(device="cuda:7", dtype=torch.bfloat16)
merger.__post_init__()
merger = merger.to(device="cuda:7", dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Initializing masks: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [01:02<00:00,  4.11it/s]
2024-12-24 09:25:00,900 - INFO - Initial GPU memory allocated: 0.00 GB
2024-12-24 09:25:03,362 - INFO - Final GPU memory allocated: 0.00 GB
2024-12-24 09:25:03,363 - INFO - Freed GPU memory: 0.00 GB


In [19]:
# def _constrain_spherical(self, mask_weights: List[torch.Tensor], DOT_THRESHOLD: float = 0.9995):
#     assert len(mask_weights) == 2, (
#         "Spherical constraint (SLERP) only supports 2 mask weights"
#     )

    
#     T = my_silu(mask_weights[0])

#     # Angle at timestep t's
#     theta_ts = self.theta_0s * T
#     sin_theta_ts = torch.sin(theta_ts)

#     # Finish calculating slerp factors
#     S0 = torch.sin(self.theta_0s - theta_ts) / self.sin_theta_0s
#     S1 = sin_theta_ts / self.sin_theta_0s

#     # Avoid NaN
#     nan_indices = self.dots.float() > DOT_THRESHOLD
#     S0[nan_indices] = 1 - T[nan_indices]
#     S1[nan_indices] = T[nan_indices]
    
#     return [S0, S1]

In [20]:
merger.merger.model.embed_tokens.masks_constrainer.constrain_mode

'spherical'

In [21]:
merger.merger.model.embed_tokens.masks_constrainer.dots

tensor(0.9844, device='cuda:7', dtype=torch.bfloat16)

In [65]:
# x = merger.merger.model.layers[10].mlp.down_proj.get_raw_masks()['weight_masks'][0].data
# cons = merger.merger.model.layers[10].mlp.down_proj.weight_masks_constrainer
# T = my_silu(x)
# T

tensor(0.5938, device='cuda:7', dtype=torch.bfloat16)

In [66]:
# theta_ts = cons.theta_0s * T
# theta_ts

tensor(0.0525, device='cuda:7', dtype=torch.bfloat16)

In [68]:
# 1 - T

tensor(0.4062, device='cuda:7', dtype=torch.bfloat16)

In [70]:
# torch.sin(cons.theta_0s - theta_ts) / cons.sin_theta_0s

tensor(0.4062, device='cuda:7', dtype=torch.bfloat16)

In [87]:
# (cons.theta_0s - theta_ts).float().cpu().numpy()

array(0.03588867, dtype=float32)

In [88]:
# torch.sin(cons.theta_0s - theta_ts).float().cpu().numpy()

array(0.03588867, dtype=float32)

In [103]:
# x = torch.tensor(0.111, dtype=torch.bfloat16)
# torch.sin(x) == x

tensor(True)

In [22]:
merger.merger.model.layers[10].mlp.down_proj.get_raw_masks()

{'weight_masks': [Parameter containing:
  tensor(1., device='cuda:7', dtype=torch.bfloat16, requires_grad=True),
  Parameter containing:
  tensor(1., device='cuda:7', dtype=torch.bfloat16, requires_grad=True)],
 'bias_masks': [None, None]}

In [39]:
merger.merger.model.layers[13].mlp.down_proj.get_constrained_masks()

{'weight_masks': [tensor(0.5039, device='cuda:7', dtype=torch.bfloat16),
  tensor(0.5039, device='cuda:7', dtype=torch.bfloat16)],
 'bias_masks': [None, None]}

In [25]:
merger.merger.model.layers[10].mlp.down_proj.weight_masks_constrainer.theta_0s

tensor(0.1533, device='cuda:7', dtype=torch.bfloat16)

In [26]:
# dots = merger.merger.model.layers[0].mlp.gate_proj.weight_masks_constrainer.dots
# dots

In [27]:
set_masks(merger.merger, strategy="random")

Setting up masks: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [00:00<00:00, 17186.46it/s]


In [32]:
set_masks(merger.merger, strategy="odd_one_out", selected_idx=1)

Setting up masks: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 327/327 [00:00<00:00, 14337.18it/s]


In [56]:
set_masks(merger.merger, strategy="uniform", factors=[0.6, 0.0])

Setting up masks: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [00:00<00:00, 21177.48it/s]


In [40]:
tokenizer = AutoTokenizer.from_pretrained(merge_config.model_paths[0])

In [46]:
tokenizer.encode("<|eot_id|>")

[128000, 128009]

In [49]:
tokenizer.eos_token_id

128001

In [57]:
rewrite_example = rewrite_smol[19]
rewrite_messages = rewrite_example["messages"]
rewrite_text = tokenizer.apply_chat_template(
    rewrite_messages[:-1], tokenize=False, add_generation_prompt=True
)

summarize_example = summarize_smol[20]
summarize_messages = summarize_example["messages"]
summarize_text = tokenizer.apply_chat_template(
    summarize_messages[:-1], tokenize=False, add_generation_prompt=True
)

In [58]:
for message in summarize_messages:
    print(f"{message['role'].upper()}: {message['content']}")
    print("---" * 30)
    
answer = generate(summarize_text, merger.merger, tokenizer, max_new_tokens=256)

SYSTEM: Provide a concise, objective summary of the input text in up to three sentences, focusing on key actions and intentions without using second or third person pronouns.
------------------------------------------------------------------------------------------
USER: Police used water cannons to disperse hundreds of demonstrators gathered Tuesday in Istanbul's main commercial street to protest the building of a road through a university forest in the Turkish capital city of Ankara. Protesters gathered in Istiklal Street and chanted anti-government slogans such as, "This is just the beginning, the resistance will continue," "A thousand greetings to the resistance," "Killer police will pay," and "Police go away, these streets are ours." The new round of protests in Istanbul erupted after the Ankara Municipality began cutting trees Friday in the Middle Eastern Technical University forest to make way for a highway development project. Police used their shields and water cannons to clea

In [59]:
for message in rewrite_messages:
    print(f"{message['role'].upper()}: {message['content']}")
    print("---" * 30)

# print(rewrite_text)
answer = generate(rewrite_text, merger.merger, tokenizer, max_new_tokens=256)

SYSTEM: You're an AI assistant for text re-writing. Rewrite the input text to make it more professional and formal while retaining its essential content.
------------------------------------------------------------------------------------------
USER: Hey Sarah,

How's it going? 😄 I was thinking it would be cool if we could do a project together where we integrate some math concepts into your visual arts classes. I've been working on a few evidence-based teaching practices and thought it could really enhance the student experience. What do you think?

Can we chat more about this? Maybe we can flesh out some ideas before the staff meeting?

Thanks!
Alex
------------------------------------------------------------------------------------------
ASSISTANT: Dear Sarah,

I hope this message finds you well. I have been contemplating a collaborative project that integrates mathematical concepts into your visual arts curriculum. I believe this interdisciplinary approach, supported by evidence-ba

In [62]:
answer = generate(rewrite_text, merger.models[0], tokenizer, max_new_tokens=256)

Dear Sarah,

I hope this message finds you well. I have been considering a potential collaboration that integrates mathematical concepts into your visual arts curriculum. My recent work has focused on evidence-based teaching practices, which I believe could significantly enrich the educational experience for our students.

I would appreciate the opportunity to discuss this idea further with you at your earliest convenience. Perhaps we could schedule a time to outline some initial thoughts before the upcoming staff meeting.

Thank you for considering this proposal.

Best regards,
Alex

P.S. I am eager to hear your thoughts on this matter.<|end_of_text|>


In [63]:
answer = generate(summarize_text, merger.models[1], tokenizer, max_new_tokens=256)

Here is a concise, objective summary of the text in three sentences:

Protesters gathered in Istanbul's Istiklal Street to demonstrate against the destruction of a university forest to build a highway, with chants and slogans calling for continued resistance. Police responded with water cannons, clearing the main street and pushing protesters into side alleys, while one demonstrator was briefly knocked unconscious. The Turkish government, led by Prime Minister Recep Tayyip Erdogan, has faced criticism for its large-scale development projects, including this latest move, which has sparked renewed protests.<|eot_id|>


In [ ]:
logits_merged = get_logits(text, merger.merger, tokenizer)
logits_0 = get_logits(text, merger.models[1], tokenizer)
torch.allclose(logits_merged, logits_0, atol=0, rtol=0)